123


In [ ]:
import numpy as np
def mycsr(arr):
  indexes = []
  data = []
  indptr = [0]
  indexes = []
  sum = 0
  for i in range(len(arr)):
    for j in range(len(arr[i])):
      if (arr[i][j] != 0):
        data = np.append(data, arr[i][j])
        indexes = np.append(indexes, j)
        sum += 1
    indptr = np.append(indptr, sum)
  return data, indexes, indptr


In [ ]:
test =[[1,2,0,0,0,4],
      [0,3,0,0,5,8],
      [9,0,7,0,7,0],
      [0,0,0,1,0,0],
      [0,0,0,0,9,9],
      [0,0,0,7,0,1]]
test_csr = mycsr(test)
print(test_csr[0])
print(test_csr[1])
print(test_csr[2])

[1. 2. 4. 3. 5. 8. 9. 7. 7. 1. 9. 9. 7. 1.]
[0. 1. 5. 1. 4. 5. 0. 2. 4. 3. 4. 5. 3. 5.]
[ 0  3  6  9 10 12 14]


In [ ]:
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix

In [ ]:
class Methods:
    @staticmethod
    def get_from_matrix(csr, i, j):  # from 0
        element = 0
        start = csr.indptr[i]
        end = csr.indptr[i + 1]

        for k in range(start, end, 1):
            if (csr.indices[k] == j):
                element = csr.data[k]
                break

        return element

    @staticmethod
    def get_lu(csr):
        lu = np.matrix(np.zeros(csr.shape))
        for k in range(csr.shape[0]):
            for j in range(k, csr.shape[0]):
                lu[k, j] = Methods.get_from_matrix(csr, k, j) - lu[k, :k] * lu[:k, j]

            for i in range(k + 1, csr.shape[0]):
                lu[i, k] = (Methods.get_from_matrix(csr, i, k) - lu[i, : k] * lu[: k, k]) / lu[k, k]

        return lu

    @staticmethod
    def solve(A, b):
      #где А - матрица в формае csr, b - вектор ответов
        lu = Methods.get_lu(A)
        y = np.matrix(np.zeros([lu.shape[0], 1]))

        for i in range(0, y.shape[0]):
            y[i, 0] = b[i, 0] - lu[i, :i] * y[:i]

        x = np.matrix(np.zeros([lu.shape[0], 1]))
        for i in range(1, y.shape[0] + 1):
            x[-i, 0] = (y[-i] - lu[-i, -i:] * x[-i:, 0]) / lu[-i, -i]

        return x
    @staticmethod
    def inverse(csr):
        inv = np.zeros(csr.shape)
        for n in range(csr.shape[0]):
            b_i = np.array(np.zeros([csr.shape[0], 1]))
            b_i[n][0] = 1
            inv[:, n] = Methods.solve(csr, b_i).reshape(1, -1)
        return inv

In [ ]:
test =[[1,2,0,0,0,4],
      [0,3,0,0,5,8],
      [9,0,7,0,7,0],
      [0,0,0,1,0,0],
      [0,0,0,0,9,9],
      [0,0,0,7,0,1]]
test_csr = csr_matrix(test)
print(Methods.get_lu(test_csr))


[[ 1.  2.  0.  0.  0.  4.]
 [ 0.  3.  0.  0.  5.  8.]
 [ 9. -6.  7.  0. 37. 12.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  9.  9.]
 [ 0.  0.  0.  7.  0.  1.]]


In [ ]:
test_inv = Methods.inverse(test_csr)
np.allclose(np.dot(test,test_inv), np.dot(test_inv, test))

True

In [ ]:
test =[[1,2,0,0,0,4],
      [0,3,0,0,5,8],
      [9,0,7,0,7,0],
      [0,0,0,1,0,0],
      [0,0,0,0,9,9],
      [0,0,0,7,0,1]]
test_csr = csr_matrix(test)
answ = np.array([[1],[4],[1],[13],[1],[1]])
print(Methods.solve(test_csr,answ))
print(np.linalg.solve(test, answ))

[[ 178.7037037 ]
 [  91.14814815]
 [-319.73015873]
 [  13.        ]
 [  90.11111111]
 [ -90.        ]]
[[ 178.7037037 ]
 [  91.14814815]
 [-319.73015873]
 [  13.        ]
 [  90.11111111]
 [ -90.        ]]


In [ ]:
test1 = mycsr([[1,2,3],
               [4,5,6],
               [1,0,1]])
print(test1)

(array([1., 2., 3., 4., 5., 6., 1., 1.]), array([0., 1., 2., 0., 1., 2., 0., 2.]), array([0, 3, 6, 8]))


In [ ]:
test_csr = mycsr([[1,2,0,0,0,4],
                  [0,3,0,0,5,8],
                  [9,0,7,0,7,0],
                  [0,0,0,1,0,0],
                  [0,0,0,0,9,9],
                  [0,0,0,7,0,1]])
print(test_csr[0])
print(test_csr[1])
print(test_csr[2])

[1. 2. 4. 3. 5. 8. 9. 7. 7. 1. 9. 9. 7. 1.]
[0. 1. 5. 1. 4. 5. 0. 2. 4. 3. 4. 5. 3. 5.]
[ 0  3  6  9 10 12 14]


In [ ]:
answ = csr_matrix(test_csr, shape=(6,6)).toarray()
print(answ)

[[1. 2. 0. 0. 0. 4.]
 [0. 3. 0. 0. 5. 8.]
 [9. 0. 7. 0. 7. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 9. 9.]
 [0. 0. 0. 7. 0. 1.]]


In [ ]:
data, indexes, indptr = mycsr(
             [[1,2,0,0,0,4],
              [0,3,0,0,5,8],
              [9,0,7,0,7,0],
              [0,0,0,1,0,0],
              [0,0,0,0,9,9],
              [0,1,1,7,1,1]])
print(data)
print(indexes)
print(indptr)

[1. 2. 4. 3. 5. 8. 9. 7. 7. 1. 9. 9. 1. 1. 7. 1. 1.]
[0. 1. 5. 1. 4. 5. 0. 2. 4. 3. 4. 5. 1. 2. 3. 4. 5.]
[ 0  3  6  9 10 12 17]


In [ ]:
def LU_decomposition(A):
    """Perform LU decomposition using the Doolittle factorisation."""

    L = np.zeros_like(A)
    U = np.zeros_like(A)
    N = np.size(A, 0)

    for k in range(N):
        L[k, k] = 1
        U[k, k] = (A[k, k] - np.dot(L[k, :k], U[:k, k])) / L[k, k]
        for j in range(k+1, N):
            U[k, j] = (A[k, j] - np.dot(L[k, :k], U[:k, j])) / L[k, k]
        for i in range(k+1, N):
            L[i, k] = (A[i, k] - np.dot(L[i, :k], U[:k, k])) / U[k, k]

    return L, U

In [ ]:
test =np.array([[1,2,0,0,0,4],
    [0,3,0,0,5,8],
    [9,0,7,0,7,0],
    [0,0,0,1,0,0],
    [0,0,0,0,9,9],
    [0,0,0,7,0,1]]) 
test2 = np.array([[1,2,3],
               [4,5,6],
               [1,0,1]])
L, U = LU_decomposition(test)
print("Upper:")
print(U)
print("Lower")
print(L)
print(L.dot(U))

Upper:
[[ 1  2  0  0  0  4]
 [ 0  3  0  0  5  8]
 [ 0  0  7  0 37 12]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  9  9]
 [ 0  0  0  0  0  1]]
Lower
[[ 1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 9 -6  1  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  7  0  1]]
[[1 2 0 0 0 4]
 [0 3 0 0 5 8]
 [9 0 7 0 7 0]
 [0 0 0 1 0 0]
 [0 0 0 0 9 9]
 [0 0 0 7 0 1]]


In [ ]:

L = np.zeros_like(test)
U = np.zeros_like(test)
val = 0
N = np.size(test, 0)
for k in range(N):
  L[k, k] = 1
  for h in range(k+1)[k:]:
    for counter in range(indptr[h+1])[indptr[h]:]:
      if int(indexes[counter]) == k:
        val = data[counter]
        break
      else:
        val = 0
  U[k, k] = (val - np.dot(L[k, :k], U[:k, k])) / L[k, k]
  for j in range(k+1, N):
    for h in range(k+1)[k:]:
      for counter in range(indptr[h+1])[indptr[h]:]:
        if int(indexes[counter]) == j:
          val = data[counter]
          break
        else:
          val = 0
    U[k, j] = (val - np.dot(L[k, :k], U[:k, j])) / L[k, k]
  for i in range(k+1, N):
    for h in range(i+1)[i:]:
      for counter in range(indptr[h+1])[indptr[h]:]:
        if int(indexes[counter]) == k:
          val = data[counter]
          break
        else:
          val = 0
    L[i, k] = (val - np.dot(L[i, :k], U[:k, k])) / U[k, k]

print("Upper:")
print(U)
print("Lower")
print(L)

Upper:
[[ 1  2  0  0  0  4]
 [ 0  3  0  0  5  8]
 [ 0  0  7  0 37 12]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  9  9]
 [ 0  0  0  0  0  1]]
Lower
[[ 1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 9 -6  1  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  7  0  1]]


In [ ]:
def lu_csr(size, data, indexes, indptr): 
  L = np.zeros((size, size), int)
  U = np.zeros((size, size), int)
  val = 0
  N = size
  for k in range(N):
    L[k, k] = 1
    for h in range(k+1)[k:]:
      for counter in range(indptr[h+1])[indptr[h]:]:
        if int(indexes[counter]) == k:
          val = data[counter]
          break
        else:
          val = 0
    U[k, k] = (val - np.dot(L[k, :k], U[:k, k])) / L[k, k]
    for j in range(k+1, N):
      for h in range(k+1)[k:]:
        for counter in range(indptr[h+1])[indptr[h]:]:
          if int(indexes[counter]) == j:
            val = data[counter]
            break
          else:
            val = 0
      U[k, j] = (val - np.dot(L[k, :k], U[:k, j])) / L[k, k]
    for i in range(k+1, N):
      for h in range(i+1)[i:]:
        for counter in range(indptr[h+1])[indptr[h]:]:
          if int(indexes[counter]) == k:
            val = data[counter]
            break
          else:
            val = 0
      L[i, k] = (val - np.dot(L[i, :k], U[:k, k])) / U[k, k]
  return U, L

In [ ]:
size = np.size(test, 0)
U, L = lu_csr(size, data, indexes, indptr)

In [ ]:
print("Upper:")
print(U)
print("Lower")
print(L)

Upper:
[[ 1  2  0  0  0  4]
 [ 0  3  0  0  5  8]
 [ 0  0  7  0 37 12]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  9  9]
 [ 0  0  0  0  0  1]]
Lower
[[ 1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 9 -6  1  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  7  0  1]]


In [ ]:
print("L x U")
print(L.dot(U))
print("Original")
print(test)

L x U
[[1 2 0 0 0 4]
 [0 3 0 0 5 8]
 [9 0 7 0 7 0]
 [0 0 0 1 0 0]
 [0 0 0 0 9 9]
 [0 0 0 7 0 1]]
Original
[[1 2 0 0 0 4]
 [0 3 0 0 5 8]
 [9 0 7 0 7 0]
 [0 0 0 1 0 0]
 [0 0 0 0 9 9]
 [0 0 0 7 0 1]]


In [ ]:
def decompose_to_LU(a):
    """Decompose matrix of coefficients to L and U matrices.
     L and U triangular matrices will be represented in a single nxn matrix.
    :param a: numpy matrix of coefficients
    :return: numpy LU matrix
    """
    # create emtpy LU-matrix
    lu_matrix = np.matrix(np.zeros([a.shape[0], a.shape[1]]))
    n = a.shape[0]

    for k in range(n):
        # calculate all residual k-row elements
        for j in range(k, n):
            lu_matrix[k, j] = a[k, j] - lu_matrix[k, :k] * lu_matrix[:k, j]
        # calculate all residual k-column elemetns
        for i in range(k + 1, n):
            lu_matrix[i, k] = (a[i, k] - lu_matrix[i, : k] * lu_matrix[: k, k]) / lu_matrix[k, k]

    return lu_matrix

In [ ]:
def solve_LU(lu_matrix, b):
    
    # get supporting vector y
    y = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(y.shape[0]):
        y[i, 0] = b[i, 0] - lu_matrix[i, :i] * y[:i]

    # get vector of answers x
    x = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(1, x.shape[0] + 1):
        x[-i, 0] = (y[-i] - lu_matrix[-i, -i:] * x[-i:, 0] )/ lu_matrix[-i, -i]

    return x

In [ ]:
 ones = np.matrix(np.diag([1.0 for i in range(6)]))

In [ ]:
a = L + U - ones
answfora = np.array([[1],[4],[1],[13],[1],[1]])
print(solve_LU(a,answfora))
print(np.linalg.solve(test, answfora))

[[ 178.7037037 ]
 [  91.14814815]
 [-319.73015873]
 [  13.        ]
 [  90.11111111]
 [ -90.        ]]
[[ 178.7037037 ]
 [  91.14814815]
 [-319.73015873]
 [  13.        ]
 [  90.11111111]
 [ -90.        ]]


In [ ]:
def pick_nonzero_row(m, k):
    while k < m.shape[0] and not m[k, k]:
        k += 1
    return k

In [ ]:
m = np.hstack((a, 
                np.matrix(np.diag([1.0 for i in range(a.shape[0])]))))
n = 6
# forward trace
for k in range(n):
  # 1) Swap k-row with one of the underlying if m[k, k] = 0
  swap_row = pick_nonzero_row(m, k)
  if swap_row != k:
    m[k, :], m[swap_row, :] = m[swap_row, :], np.copy(m[k, :])
  # 2) Make diagonal element equals to 1
  if m[k, k] != 1:
    m[k, :] *= 1 / m[k, k]
  # 3) Make all underlying elements in column equal to zero
  for row in range(k + 1, n):
    m[row, :] -= m[k, :] * m[row, k]

# backward trace
  for k in range(n - 1, 0, -1):
    for row in range(k - 1, -1, -1):
      if m[row, k]:
        # 1) Make all overlying elements equal to zero in the former identity matrix
        m[row, :] -= m[k, :] * m[row, k]
a_inv = np.hsplit(m, 2)[1]

In [ ]:
np.allclose(np.dot(a, a_inv), np.eye(a.shape[0]))

True

In [ ]:
q= np.linalg.inv(a)
print(a.dot(q))

[[ 1.00000000e+00  0.00000000e+00  6.16790569e-18 -7.10542736e-15
  -5.55111512e-17  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  -5.55111512e-17  0.00000000e+00]
 [-6.66133815e-16 -4.44089210e-16  1.00000000e+00  0.00000000e+00
   1.66533454e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]


In [ ]:
def inverse_lu(lu_matrix):
  answ= []
  for i in range(len(lu_matrix)):
    m_vector = np.array(np.zeros((len(lu_matrix),1)))
    m_vector[i][0] = 1
    answ.append(solve_LU(lu_matrix,m_vector))
  return answ

In [ ]:
def inverse_lu_f(smth):
  answ = np.array(np.zeros((len(smth),len(smth))))
  for i in range(len(smth)):
    for j in range(len(smth)):
      answ[i][j] = smth[j][i]
  return answ

In [ ]:
hm = inverse_lu(a)
b = inverse_lu_f(hm)
np.allclose(np.dot(test,b), np.eye(a.shape[0]))

True

In [ ]:
class MatrixTest:
    @staticmethod
    def hilbert(n):
        return csr_matrix(np.array([[1 / (i + j + 1) for j in range(n)] for i in range(n)]))

    @staticmethod
    def diagonal(n, k):
        result = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    result[i][j] = random.randint(-4, 0)
        for i in range(n):
            result[i][i] = - sum(result[i, :]) + 10 ** (-k)
        return csr_matrix(result)

    @staticmethod
    def F(csr_A):
        n = csr_A.shape[0]
        return csr_A.toarray().dot(np.arange(1, n + 1, 1))

    @staticmethod
    def show_error_d(k):
        errors = []
        k_list = []
        for i in range(2, k + 1):
            A = MatrixTest.diagonal(i, i)
            F = MatrixTest.F(csr_matrix(A))
            predict = Methods.solve(A, F.reshape(i, -1))
            result = np.arange(1, A.shape[0] + 1, 1)
            k_list.append(i)
            errors.append(mean_absolute_error(result, predict))
        plt.plot(k_list, errors)
        plt.ylabel('Error')
        plt.xlabel('k')
        plt.show()

    @staticmethod
    def show_error_h(n):
        errors = []
        k_list = []
        for i in range(2, n + 1):
            A = MatrixTest.hilbert(i)
            F = MatrixTest.F(csr_matrix(A))
            predict = Methods.solve(A, F.reshape(i, -1))
            result = np.arange(1, A.shape[0] + 1, 1)
            k_list.append(i)
            errors.append(mean_absolute_error(result, predict))
        plt.plot(k_list, errors)
        plt.ylabel('Error')
        plt.xlabel('k')
        plt.show()

In [ ]:
class BlockMatrix:
    def __init__(self, matrix, block_size):
        self.n = block_size
        self.matrix = [csr_matrix(matrix[0:self.n, 0:self.n]),
                       csr_matrix(matrix[0:self.n, self.n:]),
                       csr_matrix(matrix[self.n:, 0:self.n]),
                       csr_matrix(matrix[self.n:, self.n:])]

    def LU(self):
        inv_X = Methods.inverse(self.matrix[0])

        L = np.eye(self.n * 2)
        L[self.n:, 0: self.n] = self.matrix[2].toarray() * inv_X

        D = np.zeros((self.n * 2, self.n * 2))
        D[0:self.n, 0: self.n] = self.matrix[0].toarray()
        D[self.n:, self.n:] = self.matrix[3].toarray() - \
                              self.matrix[2].toarray() * inv_X * self.matrix[1].toarray()

        U = np.eye(self.n * 2)
        U[0: self.n, self.n:] = inv_X * self.matrix[1]

        return L, D, U

In [ ]:
a = BlockMatrix(np.array([
                [1,2,3,4],
                [5,6,7,8],
                [9,10,11,12],
                [14,15,16,17]]), 2)

L, D, U = a.LU()
print(L)
print(D)
print(U)

[[  1.     0.     0.     0.  ]
 [  0.     1.     0.     0.  ]
 [-13.5    5.     1.     0.  ]
 [ 17.5   -3.75   0.     1.  ]]
[[   1.     2.     0.     0. ]
 [   5.     6.     0.     0. ]
 [   0.     0.    51.5   -8. ]
 [   0.     0.  -106.5   47. ]]
[[ 1.  0. -1. -2.]
 [ 0.  1.  2.  3.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
